In [1]:
from rosemary import jpt_in_notebook, jpt_parse_args, jpt_setup; jpt_setup()
if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[1] 
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


2
Thu Jan 11 15:26:15 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   29C    P0    38W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A 

In [31]:
import glob
import argparse
import logging
import math
import os
import re
import random
from copy import deepcopy
import pyarrow # add before datasets/torch
import datasets
import torch
from functools import partial
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from datasets import load_dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm.auto import tqdm
import deepspeed
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    LlamaTokenizerFast,
    CodeLlamaTokenizerFast,
    SchedulerType,
    get_scheduler,
    GPTNeoXTokenizerFast,
    GPT2Tokenizer,
    OPTForCausalLM,
    BitsAndBytesConfig,
)
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from open_instruct.dpo_utils import dpo_loss, concatenated_forward, DataCollatorForSeq2SeqDPO

import sys
sys.path.append('../open_instruct/')
from dpo_tune import (encode_with_messages_format, 
                      save_with_accelerate, 
                      prepare_deepspeed,
                      parse_args,
                      clean_checkpoints,
                     )
logger = get_logger(__name__)


## jobs submitted in notebook inherits env variables.
cache_dir = '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/cache'
os.environ['WANDB_DIR'] = cache_dir
os.makedirs(os.environ['WANDB_DIR'], exist_ok=True)
os.environ['WANDB_MODE'] = 'offline'
os.environ['WANDB_PROJECT'] = 'mitibm'

In [18]:
resume_from_checkpoint = True
use_fast_tokenizer = True
gradient_checkpointing = True
hf_models_dir = 'results/baselines/'
max_train_steps = 10
checkpointing_steps = 3
dataloader_sampler = 'RandomSampler'


# model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; max_seq_length = 2048; abbr_model_name = 'llama-7b'
model_name_or_path = hf_models_dir+'EleutherAI/pythia-70m'; max_seq_length = 2048; abbr_model_name = 'pythia-70m'
# model_name_or_path = 'results/oi2/llama-7b_sharegptv2_ep=2'; max_seq_length = 2048; abbr_model_name = 'llama7b+sharegptv2ep2'

train_file = 'data/processed/ultrafeedback/ultrafeedback_data.jsonl'; abbr_train_file = 'ultrafeedback'
subsample_inds_file = ''
dataloader_sampler = 'SequentialSampler'


output_dirname = f"{abbr_model_name}_{abbr_train_file}"
output_dirname = 'jpt_'+output_dirname
output_dir = os.path.join('results', output_dirname)

cmd = f"""
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    {'--use_slow_tokenizer' if not  use_fast_tokenizer else ''} \
    {'--gradient_checkpointing' if gradient_checkpointing  else ''} \
    --train_file {train_file} \
    --max_seq_length {max_seq_length} \
    {'--subsample_inds_file '+subsample_inds_file if subsample_inds_file else ''} \
    --dataloader_sampler {dataloader_sampler} \
    --preprocessing_num_workers 32 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 32 \
    --learning_rate 5e-7 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.1 \
    --weight_decay 0. \
    --num_train_epochs 1 \
    --with_tracking \
    --report_to tensorboard \
    --checkpointing_steps {checkpointing_steps} \
    {'--max_train_steps '+str(max_train_steps) if max_train_steps else ''} \
    {'--resume_from_checkpoint' if resume_from_checkpoint else ''} \
    --low_cpu_mem_usage
    --logging_steps 1 \
    --output_dir {output_dir}

"""
print(' \\\n'.join([x.strip() for x in re.split(r'\s{3,}', cmd.strip())]))

args = parse_args(cmd)
args

--model_name_or_path results/baselines/EleutherAI/pythia-70m \
--tokenizer_name results/baselines/EleutherAI/pythia-70m \
--gradient_checkpointing \
--train_file data/processed/ultrafeedback/ultrafeedback_data.jsonl \
--max_seq_length 2048 \
--dataloader_sampler SequentialSampler \
--preprocessing_num_workers 32 \
--per_device_train_batch_size 1 \
--gradient_accumulation_steps 32 \
--learning_rate 5e-7 \
--lr_scheduler_type linear \
--warmup_ratio 0.1 \
--weight_decay 0. \
--num_train_epochs 1 \
--with_tracking \
--report_to tensorboard \
--checkpointing_steps 3 \
--max_train_steps 10 \
--resume_from_checkpoint \
--low_cpu_mem_usage \
--logging_steps 1 \
--output_dir results/jpt_pythia-70m_ultrafeedback


Namespace(dataset_name=None, dataset_config_name=None, train_file='data/processed/ultrafeedback/ultrafeedback_data.jsonl', model_name_or_path='results/baselines/EleutherAI/pythia-70m', config_name=None, use_lora=False, lora_rank=64, lora_alpha=16, lora_dropout=0.1, use_flash_attn=False, tokenizer_name='results/baselines/EleutherAI/pythia-70m', use_slow_tokenizer=False, max_seq_length=2048, per_device_train_batch_size=1, learning_rate=5e-07, weight_decay=0.0, num_train_epochs=1, max_train_steps=10, gradient_accumulation_steps=32, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.1, output_dir='results/jpt_pythia-70m_ultrafeedback', seed=None, preprocessing_num_workers=32, overwrite_cache=False, checkpointing_steps='3', logging_steps=1, resume_from_checkpoint=True, with_tracking=True, report_to='tensorboard', low_cpu_mem_usage=True, gradient_checkpointing=True, use_qlora=False, clip_grad_norm=-1, use_8bit_optimizer=False, beta=0.1, use_paged_optimizer=False, subsample_in

In [19]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
# If we're using tracking, we also need to initialize it here and it will by default pick up all supported trackers
# in the environment
accelerator_log_kwargs = {}

if args.with_tracking:
    accelerator_log_kwargs["log_with"] = args.report_to
    accelerator_log_kwargs["project_dir"] = args.output_dir

accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps, **accelerator_log_kwargs)
# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

if accelerator.is_main_process:
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)

accelerator.wait_for_everyone()

01/11/2024 16:25:07 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no



In [20]:

if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        args.dataset_name,
        args.dataset_config_name,
    )
else:
    data_files = {}
    dataset_args = {}
    if args.train_file is not None:
        data_files["train_prefs"] = args.train_file
    raw_datasets = load_dataset(
        "json",
        data_files=data_files,
        **dataset_args,
    )

01/11/2024 16:25:08 - WARNING - datasets.builder - Found cached dataset json (/gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/json/default-9a630b550f133bce/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [21]:

# Load pretrained model and tokenizer
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
else:
    raise ValueError(
        "You are instantiating a new config instance from scratch. This is not supported by this script."
    )


if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

loading configuration file results/baselines/EleutherAI/pythia-70m/config.json
Model config GPTNeoXConfig {
  "_name_or_path": "results/baselines/EleutherAI/pythia-70m",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file t

In [22]:
def load_model():
    if args.model_name_or_path:
        if args.use_qlora:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            device_index = accelerator.local_process_index
            device_map = {"": device_index} # force data-parallel training.
            model = AutoModelForCausalLM.from_pretrained(
                args.model_name_or_path,
                from_tf=bool(".ckpt" in args.model_name_or_path),
                config=config,
                load_in_4bit=True,
                quantization_config=bnb_config,
                device_map=device_map,
                torch_dtype=torch.bfloat16,
                use_flash_attention_2=True if args.use_flash_attn else False,
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                args.model_name_or_path,
                from_tf=bool(".ckpt" in args.model_name_or_path),
                config=config,
                low_cpu_mem_usage=args.low_cpu_mem_usage,
                use_flash_attention_2=True if args.use_flash_attn else False,
            )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForCausalLM.from_config(config)
    return model


In [23]:

model = load_model()
if not args.use_lora:
    reference_model = load_model()
else:
    reference_model = model


loading weights file results/baselines/EleutherAI/pythia-70m/model.safetensors
Generate config GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 0
}

All model checkpoint weights were used when initializing GPTNeoXForCausalLM.

All the weights of GPTNeoXForCausalLM were initialized from the model checkpoint at results/baselines/EleutherAI/pythia-70m.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPTNeoXForCausalLM for predictions without further training.
Generation config file not found, using a generation config created from the model config.
loading weights file results/baselines/EleutherAI/pythia-70m/model.safetensors
Generate config GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 0
}

All model checkpoint weights were used when initializing GPTNeoXForCausalLM.

All the weights of GPTNeoXForCausalLM were initialized from the model checkpoint at results/baselines/EleutherAI/pythia-70m.
If your task is similar to

In [24]:


# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map 
if isinstance(tokenizer, (LlamaTokenizer, LlamaTokenizerFast, CodeLlamaTokenizerFast)):
    from transformers import AddedToken
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": AddedToken("<s>", normalized=False, special=True),
        "eos_token": AddedToken("</s>", normalized=False, special=True),
        "unk_token": AddedToken("<unk>", normalized=False, special=True),
        "pad_token": AddedToken("<pad>", normalized=False, special=True),
    })
    ## wpq: for `huggyllama`/`NousResearch/Llama-2-7b-hf`, `LlamaTokenizerFast` tokenizer config not properly implemented and cannot tokenize special tokens like eos_token corretly. 
    # Need the following workaround. More details: https://github.com/huggingface/transformers/issues/23833
    def check_tokenizer_pad_properly(tokenizer):
        tokenizer_handles_pad_ok = True
        for s, s_tokenized in [
            ("Hi<s>Hey</s>sir<unk>what<pad><pad>", 
            ['▁Hi', '<s>', '▁Hey', '</s>', '▁sir', '<unk>', '▁what', '<pad>', '<pad>']),
        ]:
            if tokenizer.tokenize(s, add_special_tokens=False)!=s_tokenized:
                tokenizer_handles_pad_ok = False
        return tokenizer_handles_pad_ok
    if not check_tokenizer_pad_properly(tokenizer):
        if os.path.isdir(args.model_name_or_path):
            tmp_tok_path = os.path.join(
                os.path.dirname(args.model_name_or_path),
                os.path.basename(args.model_name_or_path)+'_fixtok')
            if not os.path.isdir(tmp_tok_path):
                tokenizer.save_pretrained(tmp_tok_path)
        else:
            from secrets import token_hex
            tmp_tok_path = f'/tmp/wpq_tok_{token_hex(16)}'
            tokenizer.save_pretrained(tmp_tok_path)
        tokenizer = AutoTokenizer.from_pretrained(tmp_tok_path, use_fast=not args.use_slow_tokenizer)
    assert(check_tokenizer_pad_properly(tokenizer))
elif isinstance(tokenizer, GPTNeoXTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens({
        "pad_token": "<pad>",
    })
    assert num_added_tokens == 1, "GPTNeoXTokenizer should only add one special token - the pad_token."
elif isinstance(tokenizer, GPT2Tokenizer) and isinstance(model, OPTForCausalLM):
    num_added_tokens = tokenizer.add_special_tokens({'unk_token': '<unk>'})

# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))
    if not args.use_lora:
        reference_model.resize_token_embeddings(len(tokenizer))

In [25]:

if args.use_lora:
    if args.use_qlora:
        model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=args.gradient_checkpointing)

    logger.info("Initializing LORA model...")
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        r=args.lora_rank, 
        lora_alpha=args.lora_alpha, 
        lora_dropout=args.lora_dropout,
        target_modules=["q_proj", "o_proj", "v_proj", "k_proj", "gate_proj", "up_proj", "down_proj"]
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    

# wpq: `use_cache=True` is incompatible with gradient checkpointing
model.config.use_cache = True if not args.gradient_checkpointing else False
if args.gradient_checkpointing:
    model.gradient_checkpointing_enable()


In [26]:

# Preprocessing the datasets.
if "prompt" in raw_datasets["train_prefs"].column_names and "completion" in raw_datasets["train_prefs"].column_names:
    raise ValueError("Sorry, prompt-completion format is not supported for DPO training.")
elif "chosen" in raw_datasets["train_prefs"].column_names and "rejected" in raw_datasets["train_prefs"].column_names:
    encode_function = partial(
        encode_with_messages_format,
        tokenizer=tokenizer,
        max_seq_length=args.max_seq_length,
    )
else:
    raise ValueError("You need to have 'chosen' and 'rejected in your column names.")


In [27]:

with accelerator.main_process_first():
    lm_datasets = raw_datasets["train_prefs"].map(
        encode_function,
        batched=False,
        num_proc=args.preprocessing_num_workers,
        remove_columns=[name for name in raw_datasets["train_prefs"].column_names if name not in ["chosen_input_ids", "chosen_labels", "chosen_attention_mask", "rejected_input_ids", "rejected_labels", "rejected_attention_mask"]],
        desc="Tokenizing and reformatting instruction data",
    )
    lm_datasets.set_format(type="pt")
    # wpq: if enforce max_seq_length properly, below should do nothing, so just remove.
    # # our thresholding mighta meant some examples have no labels, remove.
    # lm_datasets = lm_datasets.filter(lambda example: (example['chosen_labels'] != -100).any(), num_proc=8)
    # lm_datasets = lm_datasets.filter(lambda example: (example['rejected_labels'] != -100).any(), num_proc=8)

train_dataset = lm_datasets

Tokenizing and reformatting instruction data (num_proc=32):   0%|          | 0/60506 [00:00<?, ? examples/s]

In [28]:

if args.subsample_inds_file is not None:
    logger.info(f'Subsample dataset according to indices: {args.subsample_inds_file}')
    with open(args.subsample_inds_file, 'rb') as f:
        inds = pickle.load(f)['inds']
    logger.info(f'subsample_inds_file has {len(inds)} indices.')
    train_dataset = train_dataset.select(inds)
    
train_dataset

Dataset({
    features: ['chosen_input_ids', 'chosen_labels', 'chosen_attention_mask', 'rejected_input_ids', 'rejected_labels', 'rejected_attention_mask'],
    num_rows: 60506
})

In [13]:
example = train_dataset[2]

print('chosen:')
print(tokenizer.decode(example['chosen_input_ids']))
print('----'*20)
print('rejected:')
print(tokenizer.decode(example['rejected_input_ids']))


chosen:
<|user|>
People watch a soccer player in uniform make a kick on sand.
The question and answer are below.
Given the sentence "A black boy dressed in black shots and shirt kicks a soccer ball on sand." is it true that "While others watch."?
yes


I doubt the duck is talking to a hamster while a boy watches.
The question and answer are below.
Can we conclude from "A little boy at a lake watching a duck." that "The duck is talking to the hamster."?
Options:
- yes
- no
- it is not possible to tell
no


If you are looking out the ground you can't see the beautiful scenery.. So what could be the question?
Question followed by answer: Test for natural language inference.
Premise: "Guy looking out at beautiful scenery."
Hypothesis: "The guy is looking at the ground."
Is the hypothesis entailed by the premise?
Options:
- yes
- no
- it is not possible to tell
no


No all cloth and papers are used to communicate with an alien race.
The question and answer are below.
Given the sentence "A c

In [45]:

# wpq: add option to change sampler to allow for non-shuffled ordering of data
if args.dataloader_sampler == 'RandomSampler':
    sampler = RandomSampler(train_dataset)
elif args.dataloader_sampler == 'SequentialSampler':
    sampler = SequentialSampler(train_dataset)
else:
    raise ValueError(f'{args.dataloader_sampler} not supported.')

# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=DataCollatorForSeq2SeqDPO(tokenizer=tokenizer, model=model, padding="longest"),
    batch_size=args.per_device_train_batch_size,
    sampler=sampler,
)

In [46]:
loader = iter(train_dataloader)

for i, batch in enumerate(train_dataloader):
    print(i)
    print(tokenizer.decode(batch['chosen_input_ids'][0])[:50])
    print(tokenizer.decode(train_dataset[i]['chosen_input_ids'])[:50])
    
    if i == 3:
        break

0
<|user|>
Write a 1,000-word op-ed piece in a forma
<|user|>
Write a 1,000-word op-ed piece in a forma
1
<|user|>
Using a range of physical activities and 
<|user|>
Using a range of physical activities and 
2
<|user|>
People watch a soccer player in uniform m
<|user|>
People watch a soccer player in uniform m
3
<|user|>
What is the advantage of separable filter
<|user|>
What is the advantage of separable filter


In [47]:
batch = next(iter(train_dataloader))
import json
print(json.dumps({k: {'device': str(v.device), 'shape': list(v.shape)} for k, v in batch.items()}, indent=4))

{
    "chosen_input_ids": {
        "device": "cpu",
        "shape": [
            1,
            1006
        ]
    },
    "chosen_labels": {
        "device": "cpu",
        "shape": [
            1,
            1006
        ]
    },
    "chosen_attention_mask": {
        "device": "cpu",
        "shape": [
            1,
            1006
        ]
    },
    "rejected_input_ids": {
        "device": "cpu",
        "shape": [
            1,
            985
        ]
    },
    "rejected_labels": {
        "device": "cpu",
        "shape": [
            1,
            985
        ]
    },
    "rejected_attention_mask": {
        "device": "cpu",
        "shape": [
            1,
            985
        ]
    }
}


In [16]:

# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "layer_norm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
if args.use_qlora or args.use_paged_optimizer:
    from bitsandbytes.optim import AdamW
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=args.learning_rate,
        optim_bits=8 if args.use_8bit_optimizer else 32,
        is_paged=True
    )
else:
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=args.learning_rate)


In [17]:

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

# Create the learning rate scheduler.
# Note: the current accelerator.step() calls the .step() of the real scheduler for the `num_processes` times. This is because they assume 
# the user initialize the scheduler with the entire training set. In the case of data parallel training, each process only
# sees a subset (1/num_processes) of the training set. So each time the process needs to update the lr multiple times so that the total 
# number of updates in the end matches the num_training_steps here.
# Here we need to set the num_training_steps to either using the entire training set (when epochs is specified) or we need to multiply the 
# num_training_steps by num_processes so that the total number of updates matches the num_training_steps.
num_training_steps_for_scheduler = args.max_train_steps if overrode_max_train_steps else args.max_train_steps * accelerator.num_processes
lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_training_steps=num_training_steps_for_scheduler,
    num_warmup_steps=int(num_training_steps_for_scheduler * args.warmup_ratio),
)


logger.info(f"[wpq]  num_update_steps_per_epoch = {num_update_steps_per_epoch}")
logger.info(f"[wpq]  overrode_max_train_steps = {overrode_max_train_steps}")
logger.info(f"[wpq]  args.max_train_steps = {args.max_train_steps}")
logger.info(f"[wpq]  num_training_steps_for_scheduler = {num_training_steps_for_scheduler}")
logger.info(f"[wpq]  accelerator.num_processes = {accelerator.num_processes}")

lr_scheduler

01/10/2024 15:21:06 - INFO - __main__ - [wpq]  num_update_steps_per_epoch = 1891
01/10/2024 15:21:06 - INFO - __main__ - [wpq]  overrode_max_train_steps = False
01/10/2024 15:21:06 - INFO - __main__ - [wpq]  args.max_train_steps = 10
01/10/2024 15:21:06 - INFO - __main__ - [wpq]  num_training_steps_for_scheduler = 10
01/10/2024 15:21:06 - INFO - __main__ - [wpq]  accelerator.num_processes = 1


In [18]:

# Prepare everything with `accelerator`.
model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, lr_scheduler
)

if not jpt_in_notebook():
    if not args.use_lora:
        reference_model = prepare_deepspeed(accelerator, reference_model)
else:
    reference_model = reference_model.to('cuda')
    reference_model.eval()

In [19]:
# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if overrode_max_train_steps:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
# Afterwards we recalculate our number of training epochs
args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

# Figure out how many steps we should save the Accelerator states
checkpointing_steps = args.checkpointing_steps
if checkpointing_steps is not None and checkpointing_steps.isdigit():
    checkpointing_steps = int(checkpointing_steps)

# We need to initialize the trackers we use, and also store our configuration.
# The trackers initializes automatically on the main process.
if args.with_tracking:
    experiment_config = vars(args)
    # TensorBoard cannot log Enums, need the raw value
    experiment_config["lr_scheduler_type"] = experiment_config["lr_scheduler_type"].value
    accelerator.init_trackers("mitibm", experiment_config)

In [20]:

# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {args.num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")
# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

01/10/2024 15:21:08 - INFO - __main__ - ***** Running training *****
01/10/2024 15:21:08 - INFO - __main__ -   Num examples = 60506
01/10/2024 15:21:08 - INFO - __main__ -   Num Epochs = 1
01/10/2024 15:21:08 - INFO - __main__ -   Instantaneous batch size per device = 1
01/10/2024 15:21:08 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 32
01/10/2024 15:21:08 - INFO - __main__ -   Gradient Accumulation steps = 32
01/10/2024 15:21:08 - INFO - __main__ -   Total optimization steps = 10


  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
# wpq: instead of using `from_pretrained`, use accelerate's `load_state` to save model weights, optimizer state etc. reference: https://github.com/huggingface/accelerate/blob/v0.25.0/examples/by_feature/deepspeed_with_config_support.py

# Potentially load in the weights and states from a previous save
if args.resume_from_checkpoint:
    dirs = glob.glob(os.path.join(args.output_dir, 'step_*')) + \
        glob.glob(os.path.join(args.output_dir, 'epoch_*'))
    dirs = [x for x in dirs if os.path.isdir(x)]
    dirs.sort(key=os.path.getctime) # Sorts folders by date modified
    if len(dirs) == 0:
        args.resume_from_checkpoint = False
        logger.info(f"No checkpoint found in {args.output_dir}. Training from scratch.")
    else:
        checkpoint_path = dirs[-1]
        accelerator.load_state(checkpoint_path)
        accelerator.print(f"Resumed from checkpoint: {checkpoint_path}")
        # Extract `epoch_{i}` or `step_{i}`
        training_difference = os.path.splitext(os.path.basename(checkpoint_path))[0]

        if "epoch" in training_difference:
            starting_epoch = int(training_difference.replace("epoch_", "")) + 1
            resume_step = None
            completed_steps = starting_epoch * num_update_steps_per_epoch
        else:
            # need to multiply `gradient_accumulation_steps` to reflect real steps
            resume_step = (
                int(training_difference.replace("step_", ""))
                * args.gradient_accumulation_steps
            )
            starting_epoch = resume_step // len(train_dataloader)
            completed_steps = resume_step // args.gradient_accumulation_steps
            resume_step -= starting_epoch * len(train_dataloader)

01/10/2024 15:21:08 - INFO - accelerate.accelerator - Loading states from results/jpt_pythia-70m_ultrafeedback/step_3
01/10/2024 15:21:08 - INFO - accelerate.checkpointing - All model weights loaded successfully
01/10/2024 15:21:09 - INFO - accelerate.checkpointing - All optimizer states loaded successfully
01/10/2024 15:21:09 - INFO - accelerate.checkpointing - All scheduler states loaded successfully
01/10/2024 15:21:09 - INFO - accelerate.checkpointing - All random states loaded successfully
01/10/2024 15:21:09 - INFO - accelerate.accelerator - Loading in 0 custom states


Resumed from checkpoint: results/jpt_pythia-70m_ultrafeedback/step_3


In [29]:
from collections import defaultdict

for epoch in range(starting_epoch, args.num_train_epochs):
    model.train()
    total_loss = 0
    if (
        args.resume_from_checkpoint
        and epoch == starting_epoch
        and resume_step is not None
    ):
        # We skip the first `n` batches in the dataloader when resuming from a checkpoint
        active_dataloader = accelerator.skip_first_batches(
            train_dataloader, resume_step
        )
    else:
        active_dataloader = train_dataloader
    for step, batch in enumerate(active_dataloader):
        logps = defaultdict(int)
        
        # dpo forward pass & loss
        with accelerator.accumulate(model):
            policy_chosen_logps, policy_rejected_logps = concatenated_forward(model, batch)
            with torch.no_grad():
                if args.use_lora:
                    with accelerator.unwrap_model(model).disable_adapter():
                        reference_chosen_logps, reference_rejected_logps = concatenated_forward(model, batch)
                else:
                    reference_chosen_logps, reference_rejected_logps = concatenated_forward(reference_model, batch)
            losses, _, _ = dpo_loss(
                policy_chosen_logps, policy_rejected_logps, reference_chosen_logps, reference_rejected_logps, beta=args.beta)
            # TODO: metric logging          
            loss = losses.mean()
            # We keep track of the loss at each logged step
            total_loss += loss.detach().float()
            logps['logps_policy_chosen'] += policy_chosen_logps.detach().float()
            logps['logps_policy_rejected'] += policy_rejected_logps.detach().float()
            logps['logps_reference_chosen'] += reference_chosen_logps.detach().float()
            logps['logps_reference_rejected'] += reference_rejected_logps.detach().float()
            accelerator.backward(loss)
            # clip gradient norm. don't do this with deepspeed
            if accelerator.sync_gradients and args.clip_grad_norm > 0:
                accelerator.clip_grad_norm_(model.parameters(), args.clip_grad_norm)
            optimizer.step()
            optimizer.zero_grad()
            lr_scheduler.step()
        

        # Checks if the accelerator has performed an optimization step behind the scenes
        if accelerator.sync_gradients:
            progress_bar.update(1)
            completed_steps += 1
            if args.logging_steps and completed_steps % args.logging_steps == 0:
                avg_loss = accelerator.gather(total_loss).mean().item() / args.gradient_accumulation_steps / args.logging_steps
                for k in list(logps.keys()):
                    logps[k] = accelerator.gather(logps[k]).mean().item() / args.gradient_accumulation_steps                
                logger.info(f"  Step: {completed_steps}, LR: {lr_scheduler.get_last_lr()[0]}, Loss: {avg_loss}")
                if args.with_tracking:
                    accelerator.log(
                        {
                            "learning_rate": lr_scheduler.get_last_lr()[0],
                            "train_loss": avg_loss,
                            "step": completed_steps,
                            'logps_policy_chosen': logps['logps_policy_chosen'],
                            'logps_policy_rejected': logps['logps_policy_rejected'],
                            'logps_reference_chosen': logps['logps_reference_chosen'],
                            'logps_reference_rejected': logps['logps_reference_rejected'],
                        },
                        step=completed_steps,
                    )
                total_loss = 0

            if isinstance(checkpointing_steps, int):
                if completed_steps % checkpointing_steps == 0:
                    output_dir = f"step_{completed_steps}"
                    if args.output_dir is not None:
                        output_dir = os.path.join(args.output_dir, output_dir)
                    accelerator.save_state(output_dir)
                    clean_checkpoints(args.output_dir, keep_n=1)

            if completed_steps >= args.max_train_steps:
                break
        break

    if args.checkpointing_steps == "epoch":
        output_dir = f"epoch_{epoch}"
        if args.output_dir is not None:
            output_dir = os.path.join(args.output_dir, output_dir)
        saccelerator.save_state(output_dir)
        clean_checkpoints(args.output_dir, keep_n=1)
    


False

In [78]:

save_with_accelerate(accelerator, model, tokenizer, args.output_dir, args)
clean_checkpoints(args.output_dir, keep_n=0)


tokenizer config file saved in results/jpt_pythia-70m_ultrafeedback/tokenizer_config.json
Special tokens file saved in results/jpt_pythia-70m_ultrafeedback/special_tokens_map.json
Configuration saved in results/jpt_pythia-70m_ultrafeedback/config.json
Configuration saved in results/jpt_pythia-70m_ultrafeedback/generation_config.json
Model weights saved in results/jpt_pythia-70m_ultrafeedback/pytorch_model.bin


[]

In [75]:
l = [1,2,3]
l[:len(l)-1]

l[len(l)-1:]


[3]

In [49]:

input_ids = batch['chosen_input_ids']
attention_mask = batch['chosen_attention_mask']
labels=batch['chosen_labels']

chosen_output = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    labels=labels,
)



In [50]:
from dpo_utils import _get_batch_logps

# (bsz, seq_len, vocab)
all_logps = _get_batch_logps(chosen_output.logits, labels, average_log_prob=True)
print(all_logps)
all_logps = _get_batch_logps(chosen_output.logits, labels, average_log_prob=False)
print(all_logps)
print(chosen_output['loss'])



tensor([-1.5102], device='cuda:0', grad_fn=<DivBackward0>)
tensor([-1108.4854], device='cuda:0', grad_fn=<SumBackward1>)
tensor(1.5102, device='cuda:0', grad_fn=<NllLossBackward0>)


In [60]:
# chosen_output['loss'] * (batch['chosen_labels'][:, 1:] != 100).sum(-1)


# labels.shape, chosen_output.logits.shape

chosen_output['loss'] * (labels[:, 1:] != -100).sum(-1)



tensor([1108.4854], device='cuda:0', grad_fn=<MulBackward0>)

tensor(759, device='cuda:0')

In [61]:
from dpo_utils import concatenated_inputs

concatenated_batch = concatenated_inputs(batch)

In [62]:
import json
print(json.dumps({k: {'device': str(v.device), 'shape': list(v.shape)} for k, v in batch.items()}, indent=4))
print(json.dumps({k: {'device': str(v.device), 'shape': list(v.shape)} for k, v in concatenated_batch.items()}, indent=4))

{
    "chosen_input_ids": {
        "device": "cuda:0",
        "shape": [
            1,
            180
        ]
    },
    "chosen_labels": {
        "device": "cuda:0",
        "shape": [
            1,
            180
        ]
    },
    "chosen_attention_mask": {
        "device": "cuda:0",
        "shape": [
            1,
            180
        ]
    },
    "rejected_input_ids": {
        "device": "cuda:0",
        "shape": [
            1,
            170
        ]
    },
    "rejected_labels": {
        "device": "cuda:0",
        "shape": [
            1,
            170
        ]
    },
    "rejected_attention_mask": {
        "device": "cuda:0",
        "shape": [
            1,
            170
        ]
    }
}
{
    "concatenated_input_ids": {
        "device": "cuda:0",
        "shape": [
            2,
            180
        ]
    },
    "concatenated_labels": {
        "device": "cuda:0",
        "shape": [
            2,
            180
        ]
    },
    "con

In [79]:
print(policy_chosen_logps.item(), policy_rejected_logps.item())
print(reference_chosen_logps.item(), reference_rejected_logps.item())
print(losses.item())

-3264.91552734375 -480.8586120605469
-3264.8359375 -480.8492431640625
0.6966567635536194


(tensor([-3264.8359], device='cuda:0'), tensor([-480.8492], device='cuda:0'))

In [71]:
batch['chosen_input_ids'].shape[0]

1